In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [50]:
# extract product title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": "productTitle"}).text.strip()

    except AttributeError:
        title = ""

    return title


# extract product price
def get_price(soup):
    try:
        price = soup.find("span", attrs={
                          "class": "a-price aok-align-center reinventPricePriceToPayMargin priceToPay"}).find("span", attrs={"class": "a-offscreen"}).text.strip()
    except AttributeError:
        price = ""

    return price


# extract product rating
def get_rating(soup):
    try:
        rating = soup.find("div", attrs={"id": "averageCustomerReviews"}).find(
            "span", attrs={"class": "a-icon-alt"}).text.strip()
    except AttributeError:
        rating = ""

    return rating


# extract product review count
def get_review_count(soup):
    try:
        review = soup.find("span", attrs={"id": "acrCustomerReviewText"}).text
    except:
        review = ""

    return review


# extract product availablity
def get_availablity(soup):
    try:
        availablity = soup.find(
            "div", attrs={"id": "availability"}).find("span", attrs={"class": "a-size-medium a-color-success"}).text
    except AttributeError:
        availablity = ""

    return availablity

In [49]:
if __name__ == "__main__":

    # user agent : Add your user agent
    HEADERS = ({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
                "Accept-Languages": "en-US, en;q = 0.5"})

    # webpage url : amazon url
    URL = "https://www.amazon.in/s?k=playstation+5&ref=nb_sb_ss_ts-doa-p_1_3"

    # http request
    webpage = requests.get(URL, headers=HEADERS)

    # soup object contains all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # fectching links as a list of tag objects
    links = soup.find_all("a", attrs={
        "class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

    # storing the links
    links_list = []

    # extracting the links from links_list
    for link in links:
        links_list.append(link.get("href"))

    data_dict = {"title": [], "price": [],
                 "rating": [], "reviews": [], "availablity": []}

    for link in links_list:
        new_webpage = requests.get("https://amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # call and get all the necessary product data
        data_dict["title"].append(get_title(new_soup))
        data_dict["price"].append(get_price(new_soup))
        data_dict["rating"].append(get_rating(new_soup))
        data_dict["reviews"].append(get_review_count(new_soup))
        data_dict["availablity"].append(get_availablity(new_soup))

    amazon_df = pd.DataFrame.from_dict(data_dict)
    amazon_df['title'].replace("", np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_product_data.csv", header=True, index=False)


{'title': ['Sony PS5® Console – Cricket 24 Bundle', 'PS5® Console – EA SPORTS FC™ 24 Bundle', 'Sony PS5 Digital Standalone', 'Big Ant Studios Cricket 24, Standard Edition, Playstation 5', 'Sony Horizon Forbidden West | Standard Edition | PS5 Game (PlayStation 5)', "SONY PS5 GHOST OF TSUSHIMA DIRECTOR'S CUT- PS5, Black, 10GHOCU2", "Assassin's Creed Mirage | Standard Edition | PlayStation 5", 'Sony God Of War Ragnarok | Standard Edition | PS5 Game (PlayStation 5)', 'PS5 Spiderman 2 Standard Edn.', 'PlayStation 5 DualSense Wireless Controller', 'Dorca® Dust Cover for Sony Playstation 5 Cover Disc & Digital Edition', 'OIVO INDIA Hard Shell Dust Proof Cover Protective Case Anti Scratch Water-Resistant Carry-Bag for Playstation 5 Disc/Digital Version PS5 Bag (Grey)', 'Mortal Kombat 1 | Standard Edition | PlayStation 5', 'F1 23 | Standard Edition | PlayStation 5', 'New World Special Edition Charger for PS5 DualSense Controller Wireless Controller Charging Station for PS5 Controller with Dual 